此文件包含了对sd数据（来自公司员工的短信息交流数据）的coinrate 的计算

In [ ]:
%matplotlib auto
#弹出窗绘图

In [1]:
%matplotlib inline
#控制台绘图

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import pandas as pd
import numpy as np
import numba as nb
import networkx as nx

from scipy import stats
from mpl_toolkits.mplot3d import Axes3D

import random
import string
import time


---

## 运行计算函数

导入函数

In [3]:
%%time
#%%prun

%run ./coin_rate_calculate.ipynb
%run ./picture_by_matplotlib.ipynb

coin rate calculate succeed!
picture function succeed!
Wall time: 616 ms


%run ./ 当前目录

%run ../ 上一级目录

---

## 分析sd数据

### 读取数据

分析文献中的sd数据（来自公司员工的短信息交流数据）

In [4]:
pd.set_option('display.max_columns', 10,'display.max_rows', 10)  # 设置显示全部列数据行数据

In [5]:
# 读取数据
colname=['sender','recipient','timestamp']
datel=pd.read_table('.\\data\\SD01.txt',sep='\s+',header=None,names=colname,nrows=None)

In [6]:
# 获取活跃的用户的信息，发送信息大于一定值的用户
unknow=pd.value_counts(datel.loc[:,'sender'])
indx_3=unknow[unknow>500].index
print('num=%d\n'%len(indx_3),indx_3)

num=86
 Int64Index([  136,   135,    63,  1121,  1120,   812,   602,  6113,   353,
            34895,   977,    35,  1236,  8880,  1219,   603,   225,     9,
               36,   264,    80,  1218,   138,  1090,   172,   354,  1238,
              232,    79,  1346,   212,  1250, 26736, 33773,   226,  5887,
              211,   171,  8342,    85,   101,  3735,   231,  1041,  3736,
             1966,  1283,   137,   512,   102,    86,   209,  1668,   368,
              981,   514,    92,  1000,  1438,   125,   178,  3038,   145,
              840, 14729,   177,  4383,  1324,  1013,   210,  2600, 21126,
             1074,     3,  3494,   960, 27342,  1012,  1669,  1323,  5418,
             2200, 12841,  1528,  5413,  8545],
           dtype='int64')


---

In [ ]:
temp1=datel[datel.sender==136].loc[:,'timestamp']
temp2=datel[datel.sender==135].loc[:,'timestamp']
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_coin_r(600,0,temp1,temp2)

In [ ]:
pre_cr_center(600,0,temp1,temp2)

---

### 计算coinrate

In [7]:
# 生成二维空表格
neta=pd.DataFrame(np.zeros((len(indx_3),len(indx_3))),index=indx_3,columns=indx_3)
neta

,136,135,63,1121,1120,...,2200,12841,1528,5413,8545
136,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
63,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
1121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
1120,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2200,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
12841,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
1528,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
5413,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [8]:
# 二维数据中，行对象作为列对象的pre来计算,行对象与列对象的关联占行对象的比例
# 将计算得到的coinrate填充到二维空表格

for ind in indx_3:
    for col in indx_3:
        if ind==col:
            continue
        tp1=datel[datel.sender==ind].loc[:,'timestamp']
        tp2=datel[datel.sender==col].loc[:,'timestamp']
        neta.loc[ind,col]=pre_cr_(600,0,tp1,tp2)

In [9]:
# 更改表格字体颜色的准备
def showcolor(val):
    if val>0.5:
        color='red'
    elif val>0.2:
        color='green'
    else :
        color='blank'
    return 'color:%s'%color

In [ ]:
# 存储变量到其他文件中也可使用
new=neta.round(2)
%store new
%store indx_3

In [ ]:
# 更改表格的颜色并展示
neta.round(decimals=2).style.applymap(showcolor)

---

### 直接通过数据计算

In [10]:
# 直接从数据中的收发信人数据计算关系
def send_dt(sd):
    '''发信人sd，返回信息发送情况字典：sender ；rc(收信人及收信的比例)
    '''
#     发信人sd的收信人列表
    showz=datel[datel.sender==sd].loc[:,'recipient']
#     计算各个收信人的收信数量
    showzc=pd.value_counts(showz)
    rate=showzc.iloc[0]/showz.count()  #计算sender发送的信息中发给的最多的人的信息占比
    mro=showzc.index[0]
    sed_rc={'sender':sd,'rc':(showzc/showz.count())}
#     print('sender is %5d and max rate object %5d is %.3f'%(sd,mro,rate),mro in indx_3)
    return sed_rc
    

In [11]:
# 展示所有发信人联系最密切的对象
for i in indx_3:
    temp=send_dt(i)
    print(temp)

{'sender': 136, 'rc': 135     0.975743
3533    0.014408
5128    0.003648
2031    0.002553
4021    0.001824
3785    0.001094
4022    0.000547
136     0.000182
Name: recipient, dtype: float64}
{'sender': 135, 'rc': 136      0.987235
2031     0.009677
26187    0.002677
135      0.000412
Name: recipient, dtype: float64}
{'sender': 63, 'rc': 64       0.959210
38207    0.030593
2004     0.005736
44076    0.002231
2008     0.000956
7978     0.000637
29178    0.000637
Name: recipient, dtype: float64}
{'sender': 1121, 'rc': 1120     0.965686
14364    0.026395
5157     0.006410
1974     0.001131
1906     0.000377
Name: recipient, dtype: float64}
{'sender': 1120, 'rc': 1121     0.982666
16383    0.011969
31550    0.002889
17783    0.002064
1120     0.000413
Name: recipient, dtype: float64}
{'sender': 812, 'rc': 921      0.251852
4165     0.146841
3243     0.146405
2021     0.143791
813      0.096296
           ...   
4287     0.000871
4413     0.000436
4346     0.000436
43777    0.000436
42526   

{'sender': 514, 'rc': 515      0.990148
27846    0.006568
12185    0.001642
32568    0.001642
Name: recipient, dtype: float64}
{'sender': 92, 'rc': 122      0.189189
2256     0.070946
3357     0.067568
2987     0.055743
10598    0.035473
           ...   
35301    0.001689
35300    0.001689
34265    0.001689
34264    0.001689
34816    0.001689
Name: recipient, Length: 194, dtype: float64}
{'sender': 1000, 'rc': 1001     0.998305
33925    0.001695
Name: recipient, dtype: float64}
{'sender': 1438, 'rc': 1439     0.066102
2359     0.050847
4574     0.047458
5396     0.038983
1572     0.035593
           ...   
32338    0.001695
11501    0.001695
19307    0.001695
28895    0.001695
11613    0.001695
Name: recipient, Length: 145, dtype: float64}
{'sender': 125, 'rc': 126     0.897785
2650    0.100511
4546    0.001704
Name: recipient, dtype: float64}
{'sender': 178, 'rc': 177      0.762393
944      0.105983
7947     0.037607
379      0.035897
14       0.018803
33076    0.013675
19       0.01

---

### 计算coinrate得出的社交网络图

#### 设置节点和边

用箭头表示联系的流向，例如：a--->b，意味着a的信息大多发给了b

In [12]:
# 挑选coinrate的最大值
max_col_idx=np.argmax(np.array(neta),axis=0)

# 对最大值连边
max_list_edges=[]
for i in range(len(indx_3)):
    max_list_edges.append((indx_3[i],indx_3[max_col_idx[i]]))


In [20]:
print(data_5)
print(max_list_edges)

(array([ 0,  1,  3,  4,  6,  8, 11, 14, 15, 18, 20, 21, 22, 24, 25, 27, 28,
       30, 33, 34, 36, 37, 40, 41, 41, 42, 44, 44, 47, 49, 51, 52, 60, 65,
       67, 68, 69, 70, 70, 73, 74, 76, 76, 77, 78, 79, 82, 85, 85],
      dtype=int64), array([ 1,  0,  4,  3, 15, 25, 18, 21,  6, 11, 28, 14, 47, 37,  8, 42, 20,
       36,  0, 16, 30, 24, 49,  0, 44, 27,  0, 41, 22, 40, 69, 78, 65, 60,
       79, 77, 51,  0, 85, 48,  0,  0,  1, 68, 52, 67,  0,  0, 70],
      dtype=int64))
[(136, 135), (135, 136), (63, 5887), (1121, 1120), (1120, 1121), (812, 5887), (602, 603), (6113, 5887), (353, 354), (34895, 4383), (977, 27342), (35, 36), (1236, 6113), (8880, 3735), (1219, 1218), (603, 602), (225, 226), (9, 8880), (36, 35), (264, 5887), (80, 79), (1218, 1219), (138, 137), (1090, 27342), (172, 171), (354, 353), (1238, 3494), (232, 231), (79, 80), (1346, 92), (212, 211), (1250, 3494), (26736, 27342), (33773, 5887), (226, 225), (5887, 6113), (211, 212), (171, 172), (8342, 3494), (85, 5418), (101, 102), 

---

In [62]:
# 挑选大于指定数值的数的位置，第一个数是行，第二个数是列
data_5=np.where(neta>0.5)

# 对指定的数设置连边
da5_edge=[]
for i in range(len(data_5[0])):
    x1=data_5[0][i]
    x2=data_5[1][i]
    da5_edge.append((indx_3[x1],indx_3[x2]))
    print(x1,x2,'\tindex:\t',indx_3[x1],indx_3[x2])
print(da5_edge)

0 1 	index:	 136 135
1 0 	index:	 135 136
3 4 	index:	 1121 1120
4 3 	index:	 1120 1121
6 15 	index:	 602 603
8 25 	index:	 353 354
11 18 	index:	 35 36
14 21 	index:	 1219 1218
15 6 	index:	 603 602
18 11 	index:	 36 35
20 28 	index:	 80 79
21 14 	index:	 1218 1219
22 47 	index:	 138 137
24 37 	index:	 172 171
25 8 	index:	 354 353
27 42 	index:	 232 231
28 20 	index:	 79 80
30 36 	index:	 212 211
33 0 	index:	 33773 136
34 16 	index:	 226 225
36 30 	index:	 211 212
37 24 	index:	 171 172
40 49 	index:	 101 102
41 0 	index:	 3735 136
41 44 	index:	 3735 3736
42 27 	index:	 231 232
44 0 	index:	 3736 136
44 41 	index:	 3736 3735
47 22 	index:	 137 138
49 40 	index:	 102 101
51 69 	index:	 209 210
52 78 	index:	 1668 1669
60 65 	index:	 178 177
65 60 	index:	 177 178
67 79 	index:	 1324 1323
68 77 	index:	 1013 1012
69 51 	index:	 210 209
70 0 	index:	 2600 136
70 85 	index:	 2600 8545
73 48 	index:	 3 512
74 0 	index:	 3494 136
76 0 	index:	 27342 136
76 1 	index:	 27342 135
77 68 	ind

In [90]:
# 挑选大于指定数值的数的位置，第一个数是行，第二个数是列
data_5=np.where(neta>0.5)

# 对指定的数设置连边
da5_edge_re=[]
for i in range(len(data_5[0])):
    x1=data_5[0][i]
    x2=data_5[1][i]
    if x2==0 or x2==1 : continue
    da5_edge_re.append((indx_3[x1],indx_3[x2]))
    print(x1,x2,'\tindex:\t',indx_3[x1],indx_3[x2])
print(da5_edge_re)

# 设置135和136仅有一条连边
da5_edge_re.append((135,136))
da5_edge_re.append((136,135))

3 4 	index:	 1121 1120
4 3 	index:	 1120 1121
6 15 	index:	 602 603
8 25 	index:	 353 354
11 18 	index:	 35 36
14 21 	index:	 1219 1218
15 6 	index:	 603 602
18 11 	index:	 36 35
20 28 	index:	 80 79
21 14 	index:	 1218 1219
22 47 	index:	 138 137
24 37 	index:	 172 171
25 8 	index:	 354 353
27 42 	index:	 232 231
28 20 	index:	 79 80
30 36 	index:	 212 211
34 16 	index:	 226 225
36 30 	index:	 211 212
37 24 	index:	 171 172
40 49 	index:	 101 102
41 44 	index:	 3735 3736
42 27 	index:	 231 232
44 41 	index:	 3736 3735
47 22 	index:	 137 138
49 40 	index:	 102 101
51 69 	index:	 209 210
52 78 	index:	 1668 1669
60 65 	index:	 178 177
65 60 	index:	 177 178
67 79 	index:	 1324 1323
68 77 	index:	 1013 1012
69 51 	index:	 210 209
70 85 	index:	 2600 8545
73 48 	index:	 3 512
77 68 	index:	 1012 1013
78 52 	index:	 1669 1668
79 67 	index:	 1323 1324
85 70 	index:	 8545 2600
[(1121, 1120), (1120, 1121), (602, 603), (353, 354), (35, 36), (1219, 1218), (603, 602), (36, 35), (80, 79), (1218, 

---

#### 画出有向网络图

In [ ]:
# 创建有向网络图，并添加节点和连边

sd_g=nx.DiGraph()
sd_g.add_nodes_from(indx_3)
sd_g.add_edges_from(da5_edge)
# sd_g.add_edges_from(max_list_edges)

In [ ]:
# 这个是来自SD01的数据，挑选出了发送信息数大于一定值的用户,绘制出了网络图

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 有中文出现，需要用u'内容'

plt.figure(figsize=(20,20))
# text=u'节点是发送信息数大于500的用户,\n用户数是%d,coinrate>0.5就连边'%(len(indx_3))
text=u'节点是发送信息数大于500的用户,\n用户数是%d,coinrate>0.5,且去除了多连边'%(len(indx_3))

nx.draw(sd_g,
        with_labels=True,
        node_color='SpringGreen',
        edge_color='royalblue',
        node_size=500,
        width=1.5,
        arrowstyle='->',
        pos=nx.circular_layout(sd_g),        
#         pos=nx.kamada_kawai_layout(sd_g),
        )
# plt.title('network')
plt.text(0,0,text,fontdict={'color':'orangered','size':20})
# plt.savefig('.\\data\\network_SD01_u500_c_by_circular_.5.png')
# plt.savefig('.\\data\\network_SD01_u500_c_by_circular_re.png')
plt.show()

---

### 通过数据得出的社交网络图

In [22]:
# 设置节点的连边，对数据的发信和收信人直接统计，若收信人也在节点中则添加该边

edge_lists=[]
for sd in indx_3:
    senre=send_dt(sd)['rc']
    filt_d=senre[senre>0.5]
    for i in range(len(filt_d)):
        obj=filt_d.index[i]
        if obj in indx_3:
            edge_lists.append((sd,obj))
#         print(filt_d.index[i],filt_d.iloc[i])
#         print('next:\n')

In [ ]:
# 创建有向网络图，并添加节点和连边

sd_og=nx.DiGraph()
sd_og.add_nodes_from(indx_3)
sd_og.add_edges_from(edge_lists)

In [ ]:
# 这个是来自SD01的数据，挑选出了发送信息数大于一定值的用户,绘制出了网络图

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 有中文出现，需要用u'内容'

plt.figure(figsize=(20,20))
text=u'节点是发送信息数大于500的用户,\n用户数是{},接受了对方50%以上的信息的连边'.format(len(indx_3))

nx.draw(sd_og,
        with_labels=True,
        node_color='SpringGreen',
        edge_color='royalblue',
        node_size=500,
        width=1.5,
        arrowstyle='->',
        pos=nx.circular_layout(sd_og),        
#         pos=nx.kamada_kawai_layout(sd_og),
        )
# plt.title('network')
plt.text(0,0,text,fontdict={'color':'orangered','size':20})
# plt.savefig('.\\data\\network_SD01_u500_data_by_circular.png')
plt.show()

---

### 绘制网络的函数

In [ ]:
# u'节点是发送信息数大于500的用户,\n用户数是{},接受了对方50%以上的信息的连边'.format(len(indx_3))
# path='.\\data\\'+?

def pic_network(pot_no,edge_lists,txt_p=u'节点网络图',path_pic=None):
    '''绘制网络图；
    输入参数为：节点、连边；可选文字、保存路径
    '''
    # 创建有向网络图，并添加节点和连边

    sd_og=nx.DiGraph()
    sd_og.add_nodes_from(pot_no)
    sd_og.add_edges_from(edge_lists)

    # 支持中文
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    # 有中文出现，需要用u'内容'

    plt.figure(figsize=(20,20))

    nx.draw(sd_og,
            with_labels=True,
            node_color='SpringGreen',
            edge_color='royalblue',
            node_size=500,
            width=1.5,
            arrowstyle='->',
            pos=nx.circular_layout(sd_og),        
    #         pos=nx.kamada_kawai_layout(sd_og),
            )
    # plt.title('network')
    plt.text(0,0,txt_p,fontdict={'color':'orangered','size':20})
    if not path_pic==None:
        plt.savefig(path_pic)
    plt.show()

---

### 绘图

In [25]:
# 设置节点的连边，对数据的发信和收信人直接统计，若收信人也在节点中则添加该边
# 挑选了收信人中最大比例的对象

max_edge_lists=[]
for sd in indx_3:
    senre=send_dt(sd)['rc']
#     filt_d=senre[senre>0.5]
    obj=senre.index[0]
    if obj in indx_3:
        max_edge_lists.append((sd,obj))
        print((sd,obj))
        print('next:\n')

(136, 135)
next:

(135, 136)
next:

(1121, 1120)
next:

(1120, 1121)
next:

(602, 603)
next:

(353, 354)
next:

(35, 36)
next:

(1219, 1218)
next:

(603, 602)
next:

(225, 226)
next:

(36, 35)
next:

(80, 79)
next:

(1218, 1219)
next:

(138, 137)
next:

(172, 171)
next:

(354, 353)
next:

(232, 231)
next:

(79, 80)
next:

(212, 211)
next:

(26736, 27342)
next:

(226, 225)
next:

(211, 212)
next:

(171, 172)
next:

(101, 102)
next:

(3735, 3736)
next:

(231, 232)
next:

(3736, 3735)
next:

(137, 138)
next:

(512, 3)
next:

(102, 101)
next:

(209, 210)
next:

(1668, 1669)
next:

(178, 177)
next:

(177, 178)
next:

(1324, 1323)
next:

(1013, 1012)
next:

(210, 209)
next:

(2600, 8545)
next:

(3, 512)
next:

(27342, 26736)
next:

(1012, 1013)
next:

(1669, 1668)
next:

(1323, 1324)
next:

(8545, 2600)
next:



In [ ]:
txt=u'节点是发送信息数大于500的用户,\n用户数是{},接受了最多的信息的对象连边'.format(len(indx_3))
path='.\\data\\network_SD01_u500_data_by_circular_max.png'
pic_network(indx_3,edge_lists,txt_p=txt,path_pic=path)

---

### 网络的相似度

In [78]:
def similar_edge(e1,e2):
    '''计算两个连边集合的相似对度，输入两个连边的列表，输出(a&b)/(a|b)
    '''
    a1=set(e1)
    b1=set(e2)
    return len(a1 & b1)/len(a1 | b1)

In [63]:
print(len(da5_edge),'\n',da5_edge)
print(len(max_list_edges),'\n',max_list_edges)

49 
 [(136, 135), (135, 136), (1121, 1120), (1120, 1121), (602, 603), (353, 354), (35, 36), (1219, 1218), (603, 602), (36, 35), (80, 79), (1218, 1219), (138, 137), (172, 171), (354, 353), (232, 231), (79, 80), (212, 211), (33773, 136), (226, 225), (211, 212), (171, 172), (101, 102), (3735, 136), (3735, 3736), (231, 232), (3736, 136), (3736, 3735), (137, 138), (102, 101), (209, 210), (1668, 1669), (178, 177), (177, 178), (1324, 1323), (1013, 1012), (210, 209), (2600, 136), (2600, 8545), (3, 512), (3494, 136), (27342, 136), (27342, 135), (1012, 1013), (1669, 1668), (1323, 1324), (12841, 136), (8545, 136), (8545, 2600)]
86 
 [(136, 135), (135, 136), (63, 5887), (1121, 1120), (1120, 1121), (812, 5887), (602, 603), (6113, 5887), (353, 354), (34895, 4383), (977, 27342), (35, 36), (1236, 6113), (8880, 3735), (1219, 1218), (603, 602), (225, 226), (9, 8880), (36, 35), (264, 5887), (80, 79), (1218, 1219), (138, 137), (1090, 27342), (172, 171), (354, 353), (1238, 3494), (232, 231), (79, 80), (134

In [65]:
print(len(edge_lists),'\n',edge_lists)
print(len(max_edge_lists),'\n',max_edge_lists)

41 
 [(136, 135), (135, 136), (1121, 1120), (1120, 1121), (602, 603), (353, 354), (35, 36), (1219, 1218), (603, 602), (225, 226), (36, 35), (80, 79), (1218, 1219), (138, 137), (172, 171), (354, 353), (232, 231), (79, 80), (212, 211), (226, 225), (211, 212), (171, 172), (101, 102), (3735, 3736), (231, 232), (3736, 3735), (137, 138), (102, 101), (209, 210), (1668, 1669), (178, 177), (177, 178), (1324, 1323), (1013, 1012), (210, 209), (2600, 8545), (3, 512), (1012, 1013), (1669, 1668), (1323, 1324), (8545, 2600)]
44 
 [(136, 135), (135, 136), (1121, 1120), (1120, 1121), (602, 603), (353, 354), (35, 36), (1219, 1218), (603, 602), (225, 226), (36, 35), (80, 79), (1218, 1219), (138, 137), (172, 171), (354, 353), (232, 231), (79, 80), (212, 211), (26736, 27342), (226, 225), (211, 212), (171, 172), (101, 102), (3735, 3736), (231, 232), (3736, 3735), (137, 138), (512, 3), (102, 101), (209, 210), (1668, 1669), (178, 177), (177, 178), (1324, 1323), (1013, 1012), (210, 209), (2600, 8545), (3, 512)

In [97]:
a1=set(da5_edge)
a2=set(max_list_edges)
a3=set(da5_edge_re)
b1=set(edge_lists)
b2=set(max_edge_lists)

In [98]:
print(a1,'\n',a2,'\n',a3,'\n',b1,'\n',b2,'\n')

{(603, 602), (3736, 136), (3494, 136), (3, 512), (3735, 136), (178, 177), (2600, 136), (1121, 1120), (102, 101), (1120, 1121), (177, 178), (1013, 1012), (1012, 1013), (12841, 136), (27342, 135), (36, 35), (80, 79), (3736, 3735), (35, 36), (231, 232), (138, 137), (27342, 136), (1219, 1218), (1669, 1668), (135, 136), (101, 102), (1668, 1669), (354, 353), (226, 225), (137, 138), (210, 209), (8545, 136), (1218, 1219), (136, 135), (172, 171), (232, 231), (212, 211), (1324, 1323), (211, 212), (171, 172), (3735, 3736), (1323, 1324), (353, 354), (209, 210), (602, 603), (2600, 8545), (8545, 2600), (33773, 136), (79, 80)} 
 {(4383, 34895), (1120, 1121), (177, 178), (1074, 26736), (960, 5418), (26736, 27342), (35, 36), (977, 27342), (1219, 1218), (1250, 3494), (264, 5887), (354, 353), (226, 225), (210, 209), (85, 5418), (136, 135), (125, 8880), (1438, 6113), (2200, 2600), (368, 5413), (14729, 6113), (5413, 1283), (603, 602), (21126, 5887), (1013, 1012), (1012, 1013), (36, 35), (80, 79), (8342, 34

In [100]:
cr={'cr.5':da5_edge,'cr_max':max_list_edges,'cr.5_re':da5_edge_re}
da={'data.5':edge_lists,'data_max':max_edge_lists}

In [106]:
df=pd.DataFrame(np.zeros((3,2)),index=list(cr),columns=list(da))

In [111]:
for ind in list(cr):
    for col in list(da):
        df.loc[ind,col]=similar_edge(cr[ind],da[col])

In [118]:
all_data=dict(cr,**da)

In [124]:
ndf=pd.DataFrame(np.zeros((5,5)),index=list(all_data),columns=list(all_data))

In [128]:
for ind in list(all_data):
    for col in list(all_data):
#         if ind == col:
#             continue
        ndf.loc[ind,col]=similar_edge(all_data[ind],all_data[col])

In [130]:
ndf.round(decimals=2)

,cr.5,cr_max,cr.5_re,data.5,data_max
cr.5,1.00,0.42,0.82,0.80,0.75
cr_max,0.42,1.00,0.47,0.48,0.51
cr.5_re,0.82,0.47,1.00,0.98,0.91
data.5,0.80,0.48,0.98,1.00,0.93
data_max,0.75,0.51,0.91,0.93,1.00
